In [5]:
import requests
from bs4 import BeautifulSoup

In [2]:
response = requests.get('https://sociedad.info/guatemala/proveedor/28364589')

In [3]:
response.status_code

200

In [4]:
response.text

'<!DOCTYPE html><html lang=es><head><meta charset=UTF-8><meta name=viewport content="width=device-width,initial-scale=1"><link rel="shortcut icon" href=/static/img/favicon-96.png type=image/x-icon>\n<link rel=apple-touch-icon sizes=57x57 href=/static/img/favicon-96.png> <link rel=apple-touch-icon sizes=60x60 href=/static/img/favicon-96.png> <link rel=apple-touch-icon sizes=72x72 \nhref=/static/img/favicon-96.png> <link rel=apple-touch-icon sizes=76x76 href=/static/img/favicon-96.png> <link rel=apple-touch-icon sizes=114x114 href=/static/img/favicon-96.png> <link \nrel=apple-touch-icon sizes=120x120 href=/static/img/favicon-96.png> <link rel=apple-touch-icon sizes=144x144 href=/static/img/favicon-96.png> <link rel=apple-touch-icon sizes=152x152 \nhref=/static/img/favicon-96.png> <link rel=apple-touch-icon sizes=180x180 href=/static/img/favicon-96.png> <link rel=icon type=image/png sizes=192x192 href=/static/img/favicon-96.png> <link rel=icon \ntype=image/png sizes=32x32 href=/static/img

In [8]:
soup = BeautifulSoup(response.content, 'html.parser')

In [11]:
with open('../data/sociedadinfo.html', 'w') as si: 
    si.write(soup.prettify())